## 선박대기비용 절감효과(선석별)
##### BH = (V' x W' - V x W) x(WC + FC + EC)
###### 선박대기시간 예측모형 개발을 위한 방법론 연구, 2023
- BH : t년도 선박대기비용 절감효과
- Vt(Vt') : with case(without case)의 t년도 선석에서의 선박 입항 수
- Wt(Wt') : with case(without case)의 t년도 선석의 선박당 평균 대기시간 
- WC : 표준선형의 시간당 평균 체선비용
- FC : 표준선형의 시간당 평균 체화비용
- EC : 표준선형의 시간당 평균 환경비용


### 체선비용

In [105]:
import pandas as pd
PATH  = '../data/'

df = pd.read_csv(PATH + '액체_ver_2.csv')
#time_cost = pd.read_csv(PATH + '체선비용.csv', encoding = 'utf-8')
time_cost = pd.read_csv(PATH + '체선비용.csv', encoding='euc-kr')


In [106]:
time_cost['분당비용'] = time_cost['시간당비용']/60

In [107]:
time_cost

,선박크기(DWT),시간당비용,일일비용,분당비용
0,"10,000",383000,9183000,6383.333333
1,"30,000",742000,17811000,12366.666667
2,"50,000",1010000,24235000,16833.333333
3,"150,000",1959000,47006000,32650.000000
4,"200,000",2330000,55910000,38833.333333
5,"250,000",2665000,63963000,44416.666667
6,"300,000",2975000,71396000,49583.333333
7,"350,000",3265000,78351000,54416.666667
8,"400,000",3538000,84912000,58966.666667
9,"450,000",3799000,91176000,63316.666667


In [108]:
df['재화중량톤수'].describe()

count     51001.000000
mean       7633.030090
std       15886.712602
min           0.000000
25%        1250.760000
50%        2398.000000
75%        5657.000000
max      312135.000000
Name: 재화중량톤수, dtype: float64

In [109]:
time_cost['선박크기(DWT)'] = time_cost['선박크기(DWT)'].str.replace(',', '').astype(int)

In [110]:
time_cost['선박크기(DWT)']

0      10000
1      30000
2      50000
3     150000
4     200000
5     250000
6     300000
7     350000
8     400000
9     450000
10    500000
Name: 선박크기(DWT), dtype: int64

In [ ]:
bins = time_cost['선박크기(DWT)'].tolist() + [float('inf')]  # 선박크기(DWT)의 끝에 무한대를 추가하여 마지막 구간을 처리
labels = time_cost['선박크기(DWT)'].astype(str)

In [ ]:
df['DWT_구간'] = pd.cut(df['재화중량톤수'], bins=bins, labels=labels, right=False)

In [101]:
df['DWT_구간'].info()

<class 'pandas.core.series.Series'>
Int64Index: 51001 entries, 0 to 51000
Series name: DWT_구간
Non-Null Count  Dtype 
--------------  ----- 
7677 non-null   object
dtypes: object(1)
memory usage: 796.9+ KB


In [103]:
df['DWT_구간']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
50996    NaN
50997    NaN
50998    NaN
50999    NaN
51000    NaN
Name: DWT_구간, Length: 51001, dtype: object

In [102]:
time_cost['선박크기(DWT)'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 11 entries, 0 to 10
Series name: 선박크기(DWT)
Non-Null Count  Dtype
--------------  -----
11 non-null     int64
dtypes: int64(1)
memory usage: 220.0 bytes


In [81]:
merged_df = pd.merge(df, time_cost, left_on='DWT_구간', right_on='선박크기(DWT)', how='left')

In [82]:
df = merged_df.copy()

In [83]:
# df['분당_체선비용'] = df['분당비용']*df['접안_대기시간_분']

In [84]:
# df['시간당_체선비용'] = df['분당_체선비용']*60

In [100]:
df['시간당비용'].info()

<class 'pandas.core.series.Series'>
Int64Index: 51001 entries, 0 to 51000
Series name: 시간당비용
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 796.9 KB


### 시간당 체화비용
- 화물가치 x 사회적 할인율 / 365*24
- 화물가치 :  (kg당 화물가치(원) 유류 600(수입), 694(수출)의 평균값 647로 계산)
- 화물가치 = 재화중량톤수/1000 * 647

In [85]:
df['시간당_체화비용'] = df['재화중량톤수']*1000 * 647*0.045/365*24

### 환경비용

**EC 대기오염 비용**
- 배출량 구하기 ( 완료) - 온실가스 배출량(CO, NOx)
    - 2개 밖에 없음 > 한계점
- 배출량에 따른 사회적 비용 산정
    - CO/1000 * 158(온실가스 배출량에서 구한 CO)
    - NOx*15835(온실가스 배출량에서 구한 NOx)

In [86]:
df['CO비용'] = df['CO']*158
df['NOx비용'] = df['NOx']*15835

In [87]:
df['환경비용'] = df['CO비용'] + df['NOx비용']

### Year 그룹화

In [99]:
temp = df.groupby(['계선장소명', 'ETA_Year']).agg(
    입항일시=('입항일시', 'count'),
    접안_대기시간_분=('접안_대기시간_분', 'mean'),
    분당_환경비용=('환경비용', 'mean'),
    시간당_체화비용=('시간당_체화비용', 'mean'),
    시간당비용=('시간당비용', 'mean')
)

    
    
    

In [97]:
temp.reset_index(inplace = True)

In [98]:
temp.rename(columns = {'입항일시' : '입항수'}, inplace = True)

In [95]:
temp['시간당_체선비용'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 339 entries, 0 to 338
Series name: 시간당_체선비용
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 2.8 KB


### Wt(Wt') : t년도 선석의 선박당 평균 대기시간